# VR CO2 Study - Data processing

This notebook does the following:

1. x
2. y
3. z

Input: input
Output: output

In [ ]:
# TODO
#1. Participant 1 CO2 mask data time is weird

In [ ]:
# Imports
import os
if('notebooks' in os.getcwd()):
    os.chdir('..')
import json
import pandas as pd
from utils.constants import AirFiles, CO2Files, DATA_COLUMNS, FREQUENCIES
from utils.load_data import load_data_with_event_matching
from utils.timestamps import read_unix, read_j2000, j2000_to_unix, generate_biopac_unix_timestamps
from classes.Participant import Participant
from classes.DataHandler import DataHandler
from classes.Filters import Filters
import numpy as np
import matplotlib.pyplot as plt
from utils.plots import plot_eyetracking_filter


In [ ]:
# Open file index JSON for reading
file_index = pd.read_json(os.path.join(os.getcwd(), 'temp/file_index.json'))
file_index = file_index.sort_index()
participant_ids = file_index.index
participant_ids

In [ ]:
participant_ids = [1]

In [ ]:
for participant_id in participant_ids:
    participant_to_retrieve = participant_id
    participant_file_index = file_index[0][participant_to_retrieve]
    participant = Participant(participant_file_index['id'])
    
    # AIR
    print('Loading Air condition Data')
    air_mask_file = participant_file_index[AirFiles.MASK.value]
    air_event_file = participant_file_index[AirFiles.EVENT.value]
    air_eyetracking_file = participant_file_index[AirFiles.EYE.value]
    air_biopac_file = participant_file_index[AirFiles.BIOPAC.value]
    air_biopac_start_unix = participant_file_index[AirFiles.BIOPAC_UNIX_START_TIME.value]

    # Load mask data
    participant.set_air_mask_data(DataHandler.load_mask_data(air_mask_file, air_event_file, participant.id))
    # Load eye tracking data
    participant.set_air_eye_data(DataHandler.load_eyetracking_data(air_eyetracking_file, participant.id, 'air'))  
    # Load biopac data
    participant.set_air_biopac_data(DataHandler.load_biopac_data(air_biopac_file, air_biopac_start_unix, participant.id))  
    # Sync eye tracking and biopac data
    participant.set_air_synced_data(DataHandler.sync_signal_data(participant.air_mask_data, participant.air_eye_data, participant.air_biopac_data, air_biopac_start_unix))

    #CO2
    print('Loading CO2 condition Data')
    co2_mask_file = participant_file_index[CO2Files.MASK.value]
    co2_event_file = participant_file_index[CO2Files.EVENT.value]
    co2_eyetracking_file = participant_file_index[CO2Files.EYE.value]
    co2_biopac_file = participant_file_index[CO2Files.BIOPAC.value]
    co2_biopac_start_unix = participant_file_index[CO2Files.BIOPAC_UNIX_START_TIME.value]

    # Load mask data
    participant.set_co2_mask_data(DataHandler.load_mask_data(co2_mask_file, co2_event_file, participant.id))
    # Load eye tracking data
    participant.set_co2_eye_data(DataHandler.load_eyetracking_data(co2_eyetracking_file, participant.id, 'co2'))  
    # Load biopac data
    participant.set_co2_biopac_data(DataHandler.load_biopac_data(co2_biopac_file, co2_biopac_start_unix, participant.id))  
    # Sync eye tracking and biopac data
    participant.set_co2_synced_data(DataHandler.sync_signal_data(participant.co2_mask_data, participant.co2_eye_data, participant.co2_biopac_data, co2_biopac_start_unix))
    
    # Filter data
    # Normalise data
    # Feature extraction
    # ML
    

    
    print('Finished data processing for participant: ' + participant.id)



In [ ]:
#TODO: update to filter both pupils
test = Filters.filter_pupil_size(participant.air_synced_data)
plot_eyetracking_filter(participant.air_synced_data['VerboseData.Left.PupilDiameterMm'], test['VerboseData.Left.PupilDiameterMm'], participant.id, 'air')

In [ ]:
#EXAMPLES ON HOW TO RETRIEVE DATA

In [ ]:
participant.get_expression_calibration_data('air')

In [ ]:
participant.get_brightness_calibration_data('air')

In [ ]:
participant.get_condition_data('air')

In [ ]:
# Filter data with the fitstate threshold.
# POTENTIAL ISSUE WHEN ROWS WITH EVENT DATA ARE REMOVED. Should probably do it after
Filters.filter_fit_state_threshold(participant.get_expression_calibration_data('air'))

In [ ]:
# Pass participant synced data for normalisation. Different normalisation methods per modality are used
# IMPLEMENT LATER SEE BELOW
# THIS DOESNT WORK YET
test = DataHandler.normalise_data(participant.get_expression_calibration_data('air'), participant.get_brightness_calibration_data('air'), participant.get_condition_data('air'), participant.air_synced_data)




In [ ]:
from sklearn.preprocessing import MinMaxScaler
#THIS WORKS BUT MEAN FILLING IS BAD
left_eye_min_max_scaler = MinMaxScaler()
eye_tracking_df = pd.DataFrame(participant.get_brightness_calibration_data('air')[DATA_COLUMNS.EYE_LEFT_PUPIL_SIZE.value])
synced_data = pd.DataFrame(participant.air_synced_data[DATA_COLUMNS.EYE_LEFT_PUPIL_SIZE.value])
left_eye_min_max_scaler.fit(eye_tracking_df)
scaled = left_eye_min_max_scaler.transform(synced_data)
scaled = pd.DataFrame(scaled, columns=synced_data.columns, index=synced_data.index)
#left_eye_min_max_scaler = MinMaxScaler()
#left_eye_min_max_scaler.fit([participant.air_synced_data[DATA_COLUMNS.EYE_LEFT_PUPIL_SIZE.value]])
#test = left_eye_min_max_scaler.fit_transform([participant.air_synced_data[DATA_COLUMNS.EYE_LEFT_PUPIL_SIZE.value]])

In [ ]:
print(read_unix(1664462826.77676))
print(read_unix(1664462826.8157))

In [ ]:
# Convert J2000
print(read_j2000(704397173515))